In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as constants
import scipy.interpolate as interpolate
import cProfile
import glob

In [ ]:
plt.style.use("bmh")
plt.rcParams.update({
    'font.size': 12,
    'figure.figsize': (8,6),
    'xtick.minor.visible': True,
    'ytick.minor.visible': True})

In [ ]:
%autoreload 2
import madart.dualpy as dp
from madart.sample import sample

In [ ]:
import madart.trivial_l2gp as l2gp
molecules = [ "O3", "H2O", "CO", "HNO3" ]
products = [ "Temperature", ] + molecules
l2 = dict()
for product in products:
    l2[product] = l2gp.l2gp_read (
        "/data/emls/l2gp/v04.23/2019/001/MLS-Aura_L2GP-" +
        product + 
        "_v04-23-c02_2019d001.he5", 
        product )

In [ ]:
product = "Temperature"
pressure = np.array(l2[product].pressure)
nProfs = 3
zeta = -np.log10(pressure)
phi = np.array(l2[product].phi[0:nProfs])
values = np.array(l2[product][0:nProfs,:])
values = np.expand_dims(values,-1)
values = np.repeat(values,2,-1)
values[:,:,-1] += 1000.0
# offsets = np.transpose([[-30,0,30]])
# values += offsets
print(values.shape)
values = dp.seed(values, product)
print(values[:,0,:])

In [ ]:
# values = dp.seed(values,"vmr")
pOut = np.array([1000.0,900.0,100.0,90.0,10.0])
phi0 = phi[0] + 1000.0
phiOut = phi0 #+ np.array([0.0,0.75])
phiSamples, pSamples = np.meshgrid(phiOut, pOut)
zSamples = -np.log10(pSamples)
print (phiSamples)
print (pSamples)

In [ ]:
# %%timeit
test = sample(values, (phi,zeta), (phiSamples,zSamples))
print(test.shape)

In [ ]:
fig, ax = plt.subplots()
i = 1
ax.plot(np.transpose(np.squeeze(values[0:3,:,i])), np.transpose(pressure))
ax.plot(np.squeeze(test[:,:,i]), pSamples, 'o')
ax.set_yscale("Log")
ax.set_ylim(1000.0,1.0)
ax.set
plt.show()

In [ ]:
print(test.jacobians["Temperature"].data)

In [ ]:
?scipy.interpolate.splev

In [ ]:
import scipy.interpolate as interpolate

In [ ]:
?interpolate.splrep

In [ ]:
?interpolate.BSpline